## Threading in Python

### What is a thread?

- a process(separate).

### What is GIL ?

- Global Interpreter Lock

### When to use multi-threading in python?

- tasks that require I/O operations that may take a long time.

### Starting a thread

In [233]:
import time
import threading

start = time.time()


def foo():
    print('About to sleep...')
    time.sleep(2)
    print('Done sleeping...')

thread = threading.Thread(target=foo)
thread_1 = threading.Thread(target=foo)

thread.start()
thread_1.start()

end = time.time()

print(f'Time taken:{end - start:.2f}')
 

About to sleep...
About to sleep...
Time taken:0.00
Done sleeping...
Done sleeping...


### Passing values to threads

In [235]:
import threading
import time


def foo(n):
    print(f'About to sleep for {n} second(s)>>>>')
    time.sleep(n)
    print('Done sleeping...')
    

start = time.perf_counter()

thread = threading.Thread(target=foo, kwargs={'n': 2})
thread.start()

end = time.perf_counter()

print(f'Completed in {end - start:.2f}')


About to sleep for 2 second(s)>>>>
Completed in 0.00
Done sleeping...


### Starting multiple threads

In [238]:
import threading
import time


def foo(t):
    print('About to sleep for %s second(s)....' % t)
    time.sleep(t)
    print('Done sleeping for %s seconds...' % t)

    
start = time.perf_counter()
    
for t in range(1, 6):
    thread = threading.Thread(target=foo, args=[t])
    thread.start()
    
end = time.perf_counter()


print(f'Complete in: {end - start:.2f} second(s)')

About to sleep for 1 second(s)....
About to sleep for 2 second(s)....About to sleep for 3 second(s)....

About to sleep for 4 second(s)....
About to sleep for 5 second(s)....Complete in: 0.01 second(s)

Done sleeping for 1 seconds...
Done sleeping for 2 seconds...
Done sleeping for 3 seconds...
Done sleeping for 4 seconds...
Done sleeping for 5 seconds...


### Real life example

In [242]:
import threading
import os
import time

import requests

start = time.perf_counter()


def download_image(url):
    content = requests.get(url).content
    name = url.split('/')[-1]
    location = os.path.join('/', 'tmp', 'images', name)
    with open(location, 'wb') as f:
        f.write(content)


image_urls = [
    'https://www.hackadda.com/media/blog/10_Awesome_VS_Code_shortcuts.png',
    'https://www.hackadda.com/media/blog/krishna/2020/07/10/ctrl_b.gif',
    'https://www.hackadda.com/media/blog/krishna/2020/07/10/ctrl_p.gif',
    'https://www.hackadda.com/media/blog/krishna/2020/07/10/ctrl_shift_f_with_alt_c_and_alt_w.gif',
    'https://www.hackadda.com/media/blog/krishna/2020/07/10/ctrl_shift_e.gif'
]

for url in image_urls:
    thread = threading.Thread(target=download_image, args=[url])
    thread.start()
    
end = time.perf_counter()

print(f'Completed in {end - start: .2f} second(s)')

Completed in  0.02 second(s)


In [240]:
import os


os.chdir('/tmp/images')

for image in os.listdir(os.getcwd()):
    os.remove(image)

### Synchronizing Threads

In [243]:
import threading
import time


def foo(t):
    print('About to sleep for %s second(s)....' % t)
    time.sleep(t)
    print('Done sleeping for %s seconds...' % t)

    
start = time.perf_counter()
    
thread = threading.Thread(target=foo, args=[2])
thread.start()
    
thread.join()
    
end = time.perf_counter()


print(f'Complete in: {end - start:.2f} second(s)')

About to sleep for 2 second(s)....
Done sleeping for 2 seconds...
Complete in: 2.00 second(s)


### Another Exmaple

In [244]:
import threading
import time


def foo(t):
    print('About to sleep for %s second(s)....' % t)
    time.sleep(t)
    print('Done sleeping for %s seconds...' % t)

    
start = time.perf_counter()
    
threads = []
for t in range(1, 6):
    thread = threading.Thread(target=foo, args=[t])
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()
    
end = time.perf_counter()


print(f'Complete in: {end - start:.2f} second(s)')

About to sleep for 1 second(s)....
About to sleep for 2 second(s)....
About to sleep for 3 second(s)....About to sleep for 4 second(s)....

About to sleep for 5 second(s)....
Done sleeping for 1 seconds...
Done sleeping for 2 seconds...
Done sleeping for 3 seconds...
Done sleeping for 4 seconds...
Done sleeping for 5 seconds...
Complete in: 5.01 second(s)


### Newer Syntax for `threading`

In [247]:
lst = ['22', '23', '244']

list(map(int, lst))

[22, 23, 244]

In [246]:
from concurrent.futures import ThreadPoolExecutor
import time


def foo():
    print('About to sleep...')
    time.sleep(1)
    print('Awake now....')
    
    
with ThreadPoolExecutor() as executor:
#     lst = [1, 2, 3, 4, 5]
    executor.submit(foo)
    
print('Finished')

About to sleep...
Thread not completed !
Awake now....
Finished


### `Race` Condition

- different threads manipulating the same object

#### An example

In [251]:
a = 8
f'{a=}'

'a=8'

In [252]:
from concurrent.futures import ThreadPoolExecutor
import time


class Data:
    def __init__(self):
        self.val = 0
        
    def update(self, n):
        print(f'Inside thread: {n=}')
        local_val = self.val
        time.sleep(1)
        self.val = local_val + 1
    
with ThreadPoolExecutor() as executor:
    data = Data()
    for num in range(2):
        executor.submit(data.update, num)
        
print(f'value after threads are executed: {data.val=}')
        

Inside thread: n=0Inside thread: n=1

value after threads are executed: data.val=1


#### Handling `Race` condition

In [261]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading


class Data:
    def __init__(self):
        self.val = 0
        self._lock = threading.Lock()
        
    def update(self, n):
        print(f'Inside thread: {n=}')
        with self._lock:
            local_val = self.val
            print(f'Locked thread: {n=}')
            time.sleep(1)
            self.val = local_val + 1
    
with ThreadPoolExecutor() as executor:
    data = Data()
    for num in range(2):
        executor.submit(data.update, num)
        
print(f'value after threads are executed: {data.val=}')

Inside thread: n=0
Locked thread: n=0Inside thread: n=1

Locked thread: n=1
value after threads are executed: data.val=2


### Deadlock

### Real life deadlock


**Interviewer**: *What is deadlock?*

**Interviewee**: *Hire me and I will tell you what a deadlock is*

### Multithreading v/s Multiprocessing

In [259]:
from concurrent.futures import ProcessPoolExecutor


def is_prime(n):
    pass

with ProcessPoolExecutor() as executor:
    nums = []
    
    for num in nums:
        executor.submit(is_prime, nums)

SyntaxError: invalid syntax (<ipython-input-259-33cdb03cbfed>, line 1)